In [43]:
import pandas as pd

In [46]:
cruises = pd.read_csv('data/01_cruises.csv')

cruises = cruises[['Dato','ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]

# Filling the times for World Traveler
row_index = 16
column_name = 'ETA'
value_to_fill = '08:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 16
column_name = 'ETD'
value_to_fill = '17:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# Filling the times for Jewel of the Sea which stayed overnight
row_index = 47
column_name = 'ETD'
value_to_fill = '23:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 49
column_name = 'ETA'
value_to_fill = '00:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# zbavily jsme se Queen Victoria na indexu 15, ktera nedoplula.
cruises = cruises.drop(15, axis='index') 

cruises.to_csv('data/cruises.csv', index=False)



# # Kontrola toho, ze uz nemame v cruises hodnoty s nevyplnenym casem
# lode = cruises[['Navn','Passasjerer','Dato','ETA','ETD']] #timhle si vyberu, ktere tabulky maji chybu a kuzu si to zkontrolovat v tabulce s rozpisem sluzeb, kdy lode opravdu dojely

# nula = lode[lode.isna().any(axis=1)]
# nula


